### 导入依赖

In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 使用函数

In [2]:
def cv_show(img):
    cv.imshow('img',img)
    cv.waitKey(0)
    cv.destroyAllWindows()

### 导入图像

In [3]:
img = cv.imread('puff.png',0)
cv_show(img)

In [4]:
rows , cols = img . shape [0:2]

##### Image shift

In [5]:
T = np.float32 ([[1 , 0 , 50],
                 [0 , 1 , 50]])
I_shift = cv.warpAffine (img, T , (  cols , rows ))

cv_show(I_shift)

##### image flip

In [185]:
T = np.float32([[1,0,0],
               [0,-1,rows-1]])
I_flip= cv.warpAffine (img, T , (  cols , rows))
cv_show(I_flip)

In [6]:
I_flip = cv.flip(img,0)
cv_show(I_flip)

##### Uniform Image Scaling

In [8]:
scale_x,scale_y = 2,2
T = np.float32([[scale_x,0,0],
               [0,scale_y,0]])
I_uniform1=  cv.resize(img, None, fx = 2, fy = 2,interpolation = cv.INTER_CUBIC)
cv_show(I_uniform1)

In [9]:
I_uniform2= cv.warpAffine (img, T , (  cols , rows))
cv_show(I_uniform2)

##### Image Rotation

In [10]:
import math
phi = 17.0 * math.pi / 180
T = np . float32 ([[ math . cos ( phi ) , - math . sin ( phi ) , 0] ,
                    [ math . sin ( phi ) , math . cos ( phi ) , 0]])
I_rotate = cv.warpAffine (img, T , ( cols , rows ))
cv_show(I_rotate)

In [11]:
phi = 17.0
T = cv.getRotationMatrix2D( ((cols - 1) / 2.0, (rows - 1) / 2.0), -phi, 1)
I_rotate = cv.warpAffine (img, T , ( cols , rows ))
cv_show(I_rotate)

##### Affine mapping

In [12]:
pts_src = np.float32([[50, 300], [150, 200], [50, 50]])
pts_dst = np.float32([[50, 200], [250, 200], [50, 100]])
T = cv.getAffineTransform(pts_src, pts_dst)
I_mapping = cv.warpAffine (img, T , ( cols , rows ))
cv_show(I_mapping)

##### Image Bevel

In [14]:
s = 0.3
T = np . float32 ([[1 , s , 0] , [0 , 1 , 0]])
I_bevel = cv.warpAffine (img , T , ( cols , rows ))
cv_show(I_bevel)

##### Piecewise-Linear Mapping

In [16]:
stretch = 2
T = np . float32 ([[ 2 , 0 , 0] , [0 , 1 , 0]])
I_piecewiselinear = img.copy ()
I_piecewiselinear [: , int ( cols / 2): ] =cv . warpAffine (
I_piecewiselinear [: , int ( cols / 2):] ,T , ( cols - int ( cols / 2) , rows ))

cv_show(I_piecewiselinear)

### Nonelinear Mappings
#### Projection mapping

In [19]:
T = np . float32 (
[[1.1 , 0.2 , 0.00075] ,
[0.35 , 1.1 , 0.0005] ,
[0 , 0 , 1]])
I_projective = cv.warpPerspective (img , T ,( cols , rows ))
cv_show(I_projective)

In [20]:
pts_src = np.float32([[50, 461], [461, 461], [461, 50], [50, 50]])
pts_dst = np.float32([[50, 461], [461, 440], [450, 10], [100, 50]])
T = cv.getPerspectiveTransform(pts_src, pts_dst)
I_projective = cv.warpPerspective (img , T ,( cols , rows ))
cv_show(I_projective)

##### Polynomial mapping

In [21]:
T = np . array ([[0 , 0] , [1 , 0] , [0 , 1] ,
                 [0.00001 , 0] , [0.002 , 0] , [0.001 , 0]])
I_polynomial = np . zeros ( img . shape , img . dtype )
x , y = np . meshgrid ( np . arange ( cols ) , np . arange ( rows ))

# Calculate all new X and Y coordinates
xnew = np . round ( T [0 , 0] + x * T [1 , 0] +
                   y * T [2 , 0] + x * x * T [3 , 0] +
                   x * y * T [4 , 0] +
                   y * y * T [5 , 0]). astype ( np . float32 )
ynew = np . round ( T [0 , 1] + x * T [1 , 1] +
                   y * T [2 , 1] + x * x * T [3 , 1] +
                   x * y * T [4 , 1] +
                   y * y * T [5 , 1]). astype ( np . float32 )

# Calculate mask of valid indexes
mask = np . logical_and (np . logical_and ( xnew >= 0 , xnew < cols ) ,
                         np . logical_and ( ynew >= 0 , ynew < rows ))

# Apply reindexing
if img . ndim == 2:
    I_polynomial[ ynew [ mask ]. astype ( int ) ,
              xnew [ mask ]. astype ( int )] =img [ y[ mask ] , x [ mask ]]
else :
    I_polynomial[ ynew [ mask ]. astype ( int ) ,
              xnew [ mask ]. astype ( int )] =img [ y [ mask ] , x [ mask ]]
    
cv_show(I_polynomial)

#### Simusooidal distortion

In [22]:
u , v = np . meshgrid ( np . arange ( cols ) ,np . arange ( rows ))
u = u + 20 * np . sin (2 * math . pi * v / 90)
I_sinusoid =cv . remap (img , u . astype ( np . float32 ) ,
             v . astype ( np . float32 ) , cv.INTER_LINEAR )
cv_show(I_sinusoid)

#### Distortion correction
##### Barrel distortion

In [35]:
img = cv.imread('learn_berrer.jpg.png',0)
cv_show(img)
rows , cols = img . shape [0:2]

# Create mesh grid for X , Y
xi , yi = np . meshgrid ( np . arange ( cols ) ,np . arange ( rows ))
# Shift and normalize grid
xmid = cols / 2.0
ymid = rows / 2.0
xi = xi - xmid
yi = yi - ymid


# Convert to polar and do transformation
r , theta = cv . cartToPolar ( xi / xmid , yi / ymid )
F3 = 0.1
F5 = 0.12
r = r + F3 * r ** 3 + F5 * r ** 5
# Undo conversion , normalization and shift
u , v = cv.polarToCart (r , theta )
u = u * xmid + xmid ;
v = v * ymid + ymid ;

# Do remapping
I_barrel =cv.remap(img , u . astype ( np . float32 ) ,
                    v.astype ( np . float32 ) , cv.INTER_LINEAR )

#shift
# T = np.float32 ([[1 , 0 , 250],
#                  [0 , 1 , 250]])
# I_barrel = cv.warpAffine (I_barrel, T , (  cols , rows ))

cv_show(I_barrel)

##### Image Stitching

In [ ]:
fn1 = "puff_1.png"
fn2 = "puff_2.png"
 # Read an images from files
I_top = cv . imread ( fn1 , cv . IMREAD_COLOR )
I_bottom = cv . imread ( fn2 , cv . IMREAD_COLOR )
# Match template
templ_size = 10
templ = I_top [ - templ_size : , : , :]
res = cv.matchTemplate ( I_bottom , templ ,cv.TM_CCOEFF )
min_val , max_val , min_loc ,max_loc = cv.minMaxLoc ( res )

# Stitch images
I_stitch = np . zeros (( I_top . shape [0] +
                        I_bottom . shape [0] - max_loc [1] -
                        templ_size , I_top . shape [1] ,
                        I_top . shape [2]) , dtype = np . uint8 )

I_stitch [0: I_top . shape [0] , : , :] = I_top

I_stitch [ I_top . shape [0]: , : , :] = I_bottom [ max_loc [1] + templ_size :, 1: , :]

cv_show(I_stitch)

##### Automatic panorama stitching with OpenCV

In [38]:
P2 = cv.imread('P2.jpg')
P3 = cv.imread('P3.jpg')

stitcher = cv.Stitcher_create(cv.Stitcher_SCANS)
status, I_stitch = stitcher.stitch([P2,P3])

In [7]:
cv.imwrite('file.jpg',I_stitch)

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:799: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
